# Pathway Analysis Pipeline

### Intro

* This is the `R` version of the `pathview` webapp. 
* `pathview` colours nodes on KEGG network diagrams, based on input 

### Review
* **Pros**: `pathviewR` grants access to sanitized `KEGG` pathways. That's a *very* big pro.
* **Cons**: More than the pros, unfortunately:
    * I'm *still* not sure whether the input is logFC or abundance values. Some playing around with very simple 2-class examples has revealed that this doesn't make a difference. 
    * `GAGE` (automatic pathway selection functionality, which is presumably a pathway enrichment analysis method of some kind) has questionable efficacy. It appears to need many, many features (a test dataset with ~70 features yielded output pathways, but no output `.tsv` table with the associated q-values and statistics values). 
* Suggested usage: use the **joint pathway (enrichment) analysis** module on `MetaboAnalyst` to retrieve perturbed pathways, then visualize these with `pathview`. 

In [1]:
# Load library and example datasets
library("pacman")

pacman::p_load("pathview", "gage", "tidyverse", "MetaboAnalystR", "KEGGREST", "httr")
data(gse16873.d)
# Load human pathways data
data(paths.hsa)
# load demo pathway-related data, including 3 pathway ids and related plotting params
# this is in dictionary format
data(demo.paths)

source("~/maca-utils/maca-kegg-utils.R")


## Get DE metabs

To provide `pathviewR` with fold change data.

In [2]:
fn_auc_csv <- "test_dme_data.csv"
tbl0 <- call_maca_normalization(fn_auc_csv)
tbl0 <- get_de_metabs(tbl0, 0.01, "treatment", "control", 0.05)

[1] "MetaboAnalyst R objects initialized ..."
 [1] "Successfully passed sanity check!"                                                                    
 [2] "Samples are not paired."                                                                              
 [3] "2 groups were detected in samples."                                                                   
 [4] "Only English letters, numbers, underscore, hyphen and forward slash (/) are allowed."                 
 [5] "<font color=\"orange\">Other special characters or punctuations (if any) will be stripped off.</font>"
 [6] "All data values are numeric."                                                                         
 [7] "A total of 2 (0.1%) missing values were detected."                                                    
 [8] "<u>By default, these values will be replaced by a small value.</u>"                                   
 [9] "Click <b>Skip</b> button if you accept the default practice"                

Warning message:
“Duplicated column names deduplicated: 'Alanylglycine' => 'Alanylglycine_1' [59]”

[1] "MetaboAnalyst R objects initialized ..."
[1] "Loaded files from MetaboAnalyst web-server."
[1] "Loaded files from MetaboAnalyst web-server."
[1] "Loaded files from MetaboAnalyst web-server."
NULL


In [3]:
head(tbl0)

Sample,log2_fc,KEGG,HMDB,raw_p_val,adj_p_val,fc_colour
<chr>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<chr>
Alanylglycine,-0.099317851,undef,HMDB0006899,0.767106122,0.9960022,#ACACAC
2-Methylcitric acid,0.005383113,undef,HMDB0000379,0.984630576,0.9960022,#ACACAC
4-Hydroxyproline,-2.541111981,C01157,HMDB0000725,0.332612676,0.9960022,#ACACAC
"3,4-Dihydroxybenzeneacetic acid",1.058901884,C01161,HMDB0001336,0.553459759,0.9960022,#ACACAC
Adenosine triphosphate,3.122137114,C00002,HMDB0000538,0.007198896,0.9718510,#ACACAC
3-Methyladenine,-0.674902678,C00913,HMDB0011600,0.498074864,0.9960022,#ACACAC


## Get enriched pathways from MACA

In [4]:
x <- call_maca_pw_analysis("test_dme_data.csv", "dme")
tbl1 <- x[[1]]
pw_dict <- x[[2]]
pw_names <- get_kegg_pw_ref_tbl("dme")

# Manually replace some pw_names
tbl1$pw_name[tbl1$pw_name=="Fatty acid elongation in mitochondria"] <- "Fatty acid elongation"
tbl1$pw_name[tbl1$pw_name=="Glycolysis or Gluconeogenesis"] <- "Glycolysis / Gluconeogenesis"

# Join pw enrichment output with pw IDs
tbl1 <- inner_join(tbl1, pw_names, by="pw_name")


[1] "MetaboAnalyst R objects initialized ..."
[1] "Loaded files from MetaboAnalyst web-server."
[1] "Loaded files from MetaboAnalyst web-server."
[1] "Loaded files from MetaboAnalyst web-server."
 [1] "Successfully passed sanity check!"                                                                    
 [2] "Samples are not paired."                                                                              
 [3] "2 groups were detected in samples."                                                                   
 [4] "Only English letters, numbers, underscore, hyphen and forward slash (/) are allowed."                 
 [5] "<font color=\"orange\">Other special characters or punctuations (if any) will be stripped off.</font>"
 [6] "All data values are numeric."                                                                         
 [7] "A total of 2 (0.1%) missing values were detected."                                                    
 [8] "<u>By default, these values will be

In [ ]:
pw_name_ls0 <- as.vector(unlist(tbl0["pw_name"]))
pw_name_ls1 <- as.vector(unlist(tbl1["pw_name"]))

for (pw in pw_name_ls0) {
    if (pw %in% pw_name_ls1 == F) {
        print(pw)
    }
}

## Call PathviewR

* Use `pathviewR` to retrieve the sanitized KEGG graphs (which it claims to be able to). As it turns out, it can do these in `.png` format, but not the editable `.xml` format. 
* Also do data mapping, with a single named list of logFC values (names are KEGG Ids). But these will only result in coloured nodes; barcharts are better. 
* Will return a lot of `pngs` and `xmls`. 

In [5]:
# Get input compound data
# named list of logFCs, names are KEGG Id
tbl_tmp <- tbl0 %>% dplyr::select(c("KEGG", "log2_fc")) %>% filter(KEGG != "undef")
cpd_data_ls <- unlist(tbl_tmp$log2_fc)
names(cpd_data_ls) <- unlist(tbl_tmp$KEGG)

# Get list of pathway numbers
pw_num_ls <- lapply(unlist(tbl1$pw_id), function(x) {gsub("dme", "", x)})

In [ ]:
i <- 1
suffix_i <- paste0("dme", pw_num_ls[i])
pathview(cpd.data = cpd_data_ls,
         pathway.id = pw_num_ls[i], 
         species = "dme", 
         out.suffix = suffix_i,
         keys.align = "y", 
         kegg.native = T
         )

## Get `kgml` from KEGG APIs

* This kind of works, but won't give sanitized xml pathway maps to modify. The example below (Aminoacyl-tRNA Biosynthesis) is one such instance. 

In [ ]:
pw_num_ls[1]

r <- GET("http://rest.kegg.jp/get/dme00970/kgml")
print(status_code(r))

In [ ]:
writeLines(x0_text, file("test.svg"))

In [ ]:
# Write out
xml_main <- content(r, "text")
x0_text <- strsplit(xml_main, "\n")[[1]] # to modify
write(xml_main, "dme-test1.kgml")